# Equal-Weight S&P 500 Index Fund

## Introduction & Library Imports

The S&P 500 is the world's most popular stock market index. The largest fund that is benchmarked to this index is the SPDR® S&P 500® ETF Trust. It has more than US$250 billion of assets under management.

The goal of this section of the course is to create a Python script that will accept the value of your portfolio and tell you how many shares of each S&P 500 constituent you should purchase to get an equal-weight version of the index fund.

## Library Imports

The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [1]:
import numpy as np 
import pandas as pd 
import requests

## Importing Our List of Stocks

The next thing we need to do is import the constituents of the S&P 500.

These constituents change over time, so in an ideal world you would connect directly to the index provider (Standard & Poor's) and pull their real-time constituents on a regular basis.

Paying for access to the index provider's API is outside of the scope of this course. 

There's a static version of the S&P 500 constituents available here. [Click this link to download them now](https://drive.google.com/file/d/1ZJSpbY69DVckVZlO9cC6KkgfSufybcHN/view?usp=sharing). Move this file into the `starter-files` folder so it can be accessed by other files in that directory.

Now it's time to import these stocks to our Jupyter Notebook file.

In [2]:
stocks = pd.read_csv('sp_500_stocks.csv')
stocks

,Ticker
0,A
1,AAL
2,AAP
3,AAPL
4,ABBV
...,...
500,YUM
501,ZBH
502,ZBRA
503,ZION


## Acquiring an API Token

Now it's time to import our IEX Cloud API token. This is the data provider that we will be using throughout this course.

API tokens (and other sensitive information) should be stored in a `secrets.py` file that doesn't get pushed to your local Git repository. We'll be using a sandbox API token in this course, which means that the data we'll use is randomly-generated and (more importantly) has no cost associated with it.

[Click here](http://nickmccullum.com/algorithmic-trading-python/secrets.py) to download your `secrets.py` file. Move the file into the same directory as this Jupyter Notebook before proceeding.

In [3]:
from secrets import IEX_CLOUD_API_TOKEN

## Making Our First API Call

Now it's time to structure our API calls to IEX cloud. 

We need the following information from the API:

* Market capitalization for each stock
* Price of each stock



In [4]:
symbol='AAPL'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/quote?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
data

{'avgTotalVolume': 90496004,
 'calculationPrice': 'close',
 'change': 0.91,
 'changePercent': 0.00529,
 'close': 176.16,
 'closeSource': 'iofacilf',
 'closeTime': 1646746399054,
 'companyName': 'Apple Inc',
 'currency': 'USD',
 'delayedPrice': 179.02,
 'delayedPriceTime': 1690094294927,
 'extendedChange': 0.22,
 'extendedChangePercent': 0.00126,
 'extendedPrice': 175.37,
 'extendedPriceTime': 1643199513177,
 'high': 177.48,
 'highSource': ' e5tlaryein  edcmudpei1',
 'highTime': 1690647824525,
 'iexAskPrice': None,
 'iexAskSize': None,
 'iexBidPrice': None,
 'iexBidSize': None,
 'iexClose': 177.49,
 'iexCloseTime': 1649014405777,
 'iexLastUpdated': None,
 'iexMarketPercent': None,
 'iexOpen': 179.64,
 'iexOpenTime': 1722976684853,
 'iexRealtimePrice': None,
 'iexRealtimeSize': None,
 'iexVolume': None,
 'lastTradeTime': 1696708140264,
 'latestPrice': 175.73,
 'latestSource': 'Close',
 'latestTime': 'January 14, 2022',
 'latestUpdate': 1674859032906,
 'latestVolume': 82572730,
 'low': 17

## Parsing Our API Call

The API call that we executed in the last code block contains all of the information required to build our equal-weight S&P 500 strategy. 

With that said, the data isn't in a proper format yet. We need to parse it first.

In [5]:
data['latestPrice']
data['marketCap']

2955982102948

## Adding Our Stocks Data to a Pandas DataFrame

The next thing we need to do is add our stock's price and market capitalization to a pandas DataFrame. Think of a DataFrame like the Python version of a spreadsheet. It stores tabular data.

In [6]:
my_columns = ['Ticker', 'Price','Market Capitalization']
final_dataframe = pd.DataFrame(columns = my_columns)
final_dataframe

,Ticker,Price,Market Capitalization


In [7]:
final_dataframe = final_dataframe.append(
                                        pd.Series(['AAPL', 
                                                   data['latestPrice'], 
                                                   data['marketCap'], 
                                                   ], 
                                                  index = my_columns), 
                                        ignore_index = True)
final_dataframe

,Ticker,Price,Market Capitalization
0,AAPL,175.73,2955982102948


## Looping Through The Tickers in Our List of Stocks

Using the same logic that we outlined above, we can pull data for all S&P 500 stocks and store their data in the DataFrame using a `for` loop.

In [8]:
final_dataframe = pd.DataFrame(columns = my_columns)
for symbol in stocks['Ticker'][:5]:
    print(symbol)
    api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/quote?token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(api_url).json()
    final_dataframe = final_dataframe.append(
                                        pd.Series([symbol, 
                                                   data['latestPrice'], 
                                                   data['marketCap']], 
                                                  index = my_columns), 
                                        ignore_index = True)

A
AAL
AAP
AAPL
ABBV


In [9]:
final_dataframe

,Ticker,Price,Market Capitalization
0,A,146.62,43898426918
1,AAL,19.04,12382098947
2,AAP,241.50,15607113717
3,AAPL,180.24,2916686627676
4,ABBV,140.03,250981912085


# Google Sheets

## Importing libraries

In [10]:
import gspread
from oauth2client.service_account import ServiceAccountCredentials
from pandas.io.json import json_normalize

## Connect to our service account


In [11]:
scope = ['https://spreadsheets.google.com/feeds']
credentials = ServiceAccountCredentials.from_json_keyfile_name('./jupytergooglesheets.json', scope)
gc = gspread.authorize(credentials)


In [12]:
##Get candidate data sheet from Google Drive
spreadsheet_key = '078ba185981bedaa8eaa1e862e356fd19982c328'
book = gc.open_by_key(spreadsheet_key)
worksheet = book.worksheet("Candidate Data")
table = worksheet.get_all_values()
table[:8]

APIError: {'code': 403, 'message': 'Google Sheets API has not been used in project 518783657123 before or it is disabled. Enable it by visiting https://console.developers.google.com/apis/api/sheets.googleapis.com/overview?project=518783657123 then retry. If you enabled this API recently, wait a few minutes for the action to propagate to our systems and retry.', 'status': 'PERMISSION_DENIED', 'details': [{'@type': 'type.googleapis.com/google.rpc.Help', 'links': [{'description': 'Google developers console API activation', 'url': 'https://console.developers.google.com/apis/api/sheets.googleapis.com/overview?project=518783657123'}]}, {'@type': 'type.googleapis.com/google.rpc.ErrorInfo', 'reason': 'SERVICE_DISABLED', 'domain': 'googleapis.com', 'metadata': {'service': 'sheets.googleapis.com', 'consumer': 'projects/518783657123'}}]}